# Ahorcado

El código desarrollado a continuación simula el juego del ahorcado tomando una base de datos (archivo bdd_ahorcado.csv), únicamente se debe indicar la ruta donde vive este archivo en la variable path de la sección de prueba.

Las categorías disponibles de palabras a descubrir son cartas de la lotería mexicana y pokemon.

### Importación de librerías

In [1]:
# Importación de librerías
import csv
import random

### Definición de funciones

In [2]:
# Diccionario que guardará las categorías con sus respectivos elementos
def gen_dic_cat(path):
    dicc_words = {}
    with open(path, newline='') as File:  
        reader = csv.reader(File)
        for row in reader:
            dicc_words.update({limpiar_acentos(row[1]): row[0]})
    # Obtener tipos de categorías
    lst_categorias = [i for i in set(dicc_words.values())]

    return dicc_words, lst_categorias


In [3]:
def welcome_instucc():
    """Función que imprime en pantalla el mensaje de bienvenida y da las instrucciones al jugador"""
    
    print('----------------------------------------------------------------------------------------------------------')
    print('Bienvenido al juego del ahorcado! :)')
    print('')
    print('Instrucciones:  El juego consiste en descubrir la palabra de acuerdo a la categoría ingresando una ')
    print('letra, sólo tienes permitido ingresar un máximo de 6 letras incorrectas, si obtienes los 6 errores')
    print('antes de descubrir la palabra pierdes el juego :( ')
    print('')
    print('Comenzamos...')
    print('')


In [4]:
def user_category(lst_categorias):
    """Función que recibe la categoría ingresada por el usuario 
    y valida que sea una categoría correcta o en defecto da el juego por terminado.
    regresa la variable de categoria"""
    
    # Imprime las categorías en pantalla
    print('Ingresa una categoría de las mostradas en pantalla:')
    for i in lst_categorias:
        print('  ', i)
    print('----------------------------------------------------------------------------------------------------------')
    print('')
    
    # Recibe la categoría ingresada por el jugador
    categoria = input()
    categoria = limpiar_acentos(categoria)
    # Valida que la categoría exista, en caso de que no pide al usuario ingresar la categoría nuevamente o
    # terminar el juego si ingresa la palabra "Salir"
    
    if categoria not in [limpiar_acentos(x) for x in lst_categorias]:
        while (categoria not in [limpiar_acentos(x) for x in lst_categorias]) and (categoria != limpiar_acentos('Salir')):
            print('Categoría incorrecta, ingresa una categoría de las mostradas en pantalla o escribe "Salir" para salir del juego')
            print('----------------------------------------------------------------------------------------------------------')
            categoria = input()
            categoria = limpiar_acentos(categoria)
            print('----------------------------------------------------------------------------------------------------------')
    
    # Imprime mensajes 
    if categoria in [limpiar_acentos(x) for x in lst_categorias]:
        print('Categoría ingresada correctamente!')
        print('----------------------------------------------------------------------------------------------------------')
    else:
        print('Juego terminado...')
        print('----------------------------------------------------------------------------------------------------------')
    return categoria

In [5]:
def count_caracteres(word, c):
    """Función que recibe un string y cuenta el número de caracteres sin tomar en cuenta el caracter c ingresado, 
    regresa el contador así como una lista con indices donde se encuentra el caracter c en el string"""
    n = 0
    lst_c=[]
    for i in range(len(word)):
        if word[i] != c:
            n+=1
        else:
            lst_c.append(i)
    return n, lst_c

def replace_caracter(lst, lst_index, new_value):
    """ Función que recibe una lista (lst) con caracteres, otra lista (lst_index) con los indices en donde se 
    encuentra el caracter que se quiere reemplazar por new_value, regresa la lista lst"""
    
    for i in lst_index:
        lst[int(i)] = new_value
    return lst

In [6]:
def word_secret(categoria, dicc_words):
    """ Función que genera la palabra secreta según la categoría y retorna 2 listas, la que contiene la palabra secreta
    separada por caracter y la lista vacía para guardar los caracteres que ingrese el usuario"""
    
    # Escoger aleatoriamente una palabra 
    lst_cat_word = [k for k,v in dicc_words.items() if limpiar_acentos(v) == categoria]
    secret_word = lst_cat_word[random.randint(0, len(lst_cat_word))]
    
    # Guardar cada caracter de secret word en una lista
    lst_word_secret = [secret_word[i] for i in range(len(secret_word)) ]
    
    # Generar una lista con guiones bajos que contenga la misma longitud que la palabra secreta
    lst_player = ['_']*len(lst_word_secret)
    
    # obtenemos la longitd y la lst_nulls con los indices donde haya espacios
    n, lst_nulls = count_caracteres(secret_word, ' ')
    
    # Imprimir la longitud de la palabra sin contar espacios
    print('La palabra a descubrir tiene ', n, 'letras')
    
    # arrglamos la lista de player
    lst_player = replace_caracter(lst_player, lst_nulls, ' ')
    
    # Mostramos en pantalla
    print('\t\t','   '.join(lst_player))
    print('')
    return secret_word, lst_word_secret, lst_player, n



In [7]:
def limpiar_acentos(text):
    acentos = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u', 'Á': 'A', 'E': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U'}
    for acen in acentos:
        if acen in text:
            text = text.replace(acen, acentos[acen])
    return text.lower()

def input_letter(intentos, lst_player, lst_word_secret, n):
    print('----------------------------------------------------------------------------------------------------------')
    letter = input('Ingresa una letra: ',  )
    if letter.isalpha() != True:        
        while letter.isalpha() != True:
            print('El valor que ingresaste no es una letra. Por favor, ingresa una letra')
            print('----------------------------------------------------------------------------------------------------------')
            letter = input()
            print('----------------------------------------------------------------------------------------------------------')
    letter = limpiar_acentos(letter)
    print('')

    if (letter in lst_word_secret) and (letter not in lst_player):
        for i in range(len(lst_word_secret)):
            if lst_word_secret[i] == letter:
                lst_player[i] = letter
        print('¡Bien hecho! :) \n')
    elif letter in lst_player:
        print('Oh, no! Esa letra ya la habías ingresado :( \n')
        intentos+=1        
    else:
        print('Oh, no! Esa letra no está en la palabra secreta :( \n')
        intentos+=1
        
    # Letras que faltan por descubir
    a=print_ahorcado(intentos)
    letters_lost = n - sum([1 for x in lst_player if x not in [' ', '_'] ])
    print('\t\t','   '.join(lst_player))
    print('')
    print('Errores:', intentos, '  |  letras por adivinar:', letters_lost)
    print('')
    return intentos, lst_player, letters_lost


def imprimir_score(intentos, lst_word_secret):
    if intentos >= 6:
        print('Has perdido, alcanzaste el máximo número de errores. La palabra era...\n')
        print('\t\t','   '.join(lst_word_secret))
    else:
        print('¡¡¡F e l i c i d a d e s!!!     Descubriste la palabra secreta\n') 
        print('\t\t','   '.join(lst_word_secret))
        
def try_again():
    print('----------------------------------------------------------------------------------------------------------')
    resp = input('¿Quieres jugar otra vez?   Si   No   ', )
    resp = resp.lower()
    
    if resp not in ['si', 'no']:
        while (resp not in ['si', 'no']):
            print('----------------------------------------------------------------------------------------------------------')
            print('Ingresa una respuesta valida')
            resp = input()
            resp = resp.lower()
            print('----------------------------------------------------------------------------------------------------------')
    if resp == 'no':
        print('Muchas gracias por jugar! :)')
    else:
        print('Vamos de nuevo! :)')
        game_ahorcado()
             

In [8]:
def print_ahorcado(n):
    if n==0:
        print('__________')
        print(' |         |')
        print(' |         ')
        print(' |        ')
        print(' |        ')
        print(' |')
        print('')
    elif n==1:
        print('__________')
        print(' |         |')
        print(' |         0')
        print(' |        ')
        print(' |        ')
        print(' |')
        print('')
    elif n==2:
        print('__________')
        print(' |         |')
        print(' |         0')
        print(' |         | ')
        print(' |          ')
        print(' |')
        print('')
    elif n==3:
        print('__________')
        print(' |         |')
        print(' |         0')
        print(' |        /| ')
        print(' |          ')
        print(' |')
        print('')
    elif n==4:
        print('__________')
        print(' |         |')
        print(' |         0')
        print(' |        /|\ ')
        print(' |         ')
        print(' |')
        print('')
    elif n==5:
        print('__________')
        print(' |         |')
        print(' |         0')
        print(' |        /|\ ')
        print(' |        /  ')
        print(' |')
        print('')
    elif n==6:
        print('__________')
        print(' |         |')
        print(' |         0')
        print(' |        /|\ ')
        print(' |        / \ ')
        print(' |')
        print('')        

In [9]:
def game_ahorcado(path):

    dicc_words, lst_categorias = gen_dic_cat(path)
    
    welcome_instucc()

    categoria = user_category(lst_categorias)
    
    if categoria.lower() != 'salir':

        secret_word, lst_word_secret,  lst_player, n = word_secret(categoria, dicc_words)

        intentos = 0
        letters_lost=n

        # Iteraar juego hasta que intentos = 5 ó letters_lost = 0
        while (intentos <6) and (letters_lost>0):
                intentos, lst_player, letters_lost = input_letter(intentos, lst_player, lst_word_secret, n)
        print('----------------------------------------------------------------------------------------------------------')

        imprimir_score(intentos, lst_word_secret)

        try_again()

### Prueba

In [10]:
path='C:/Users/Itzel/Documents/bdd_ahorcado.csv'
game_ahorcado(path)

----------------------------------------------------------------------------------------------------------
Bienvenido al juego del ahorcado! :)

Instrucciones:  El juego consiste en descubrir la palabra de acuerdo a la categoría ingresando una 
letra, sólo tienes permitido ingresar un máximo de 6 letras incorrectas, si obtienes los 6 errores
antes de descubrir la palabra pierdes el juego :( 

Comenzamos...

Ingresa una categoría de las mostradas en pantalla:
   Pokemon
   Lotería Mexicana
----------------------------------------------------------------------------------------------------------

Loteria mexicana
Categoría ingresada correctamente!
----------------------------------------------------------------------------------------------------------
La palabra a descubrir tiene  9 letras
		 _   _       _   _   _   _   _   _   _

----------------------------------------------------------------------------------------------------------
Ingresa una letra: a

¡Bien hecho! :) 

__________
